Importing Packages

In [60]:
import os
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

Initializing Dataframe

In [35]:
df = pd.DataFrame()

Setting up Dataset directory

In [36]:
data_directory = './dataset'
data_files = np.array(os.listdir(data_directory))

Listing out the Userstories in text documents

In [46]:
sentences = list()
for i in os.listdir(data_directory):
    path = os.path.join(data_directory,i)
    f = open(path,'r')
    texts = f.readlines()
    for line in texts:
        sentences.append(line.strip())
sentences[0]

' I want to have the 12-19-2017 deletions processed.'

Extract User Roles from the dataset

In [38]:
pattern = r'As (?:a|an) ([^,]+)'
roles = list()
subword_pattern = r'^.*?,'
for i in range(0,len(sentences)):
    match = re.findall(pattern,sentences[i])
    if match:
        roles.append(match[0])
    else:
        roles.append("None")
    sentences[i] = re.sub( subword_pattern,'',sentences[i])
df['role'] = roles
df['user stories'] = sentences

Dropping out the rows which has no user stories

In [44]:
df.drop(df[df['role']=="None"].index,inplace=True)

Converting to a Excel sheet

In [45]:
df.to_excel('training.xlsx', index=False)

End

Reading data from the excel

In [95]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [84]:
df = pd.read_excel('training.xlsx')

Data Preprocessing

In [93]:
def clean_text(texts):
    texts = texts.lower() # convert to lower case
    # Remove punctuation
    pattern = r'[^\w]+'
    words = texts.split() #tokenize
    for k in range(len(words)):
        words[k] = re.sub(pattern, '', words[k])
    
    texts = ' '.join(words) # Rejoin words with space
    texts = re.sub(r'\s+', ' ', texts) # Remove extra whitespace
    
    pattern = r'\b(?:\d{4}-\d{2}-\d{2}|\d{2}-\d{2}-\d{4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2},?\s\d{4}|\w+)\b'
    match = re.findall(pattern, texts) #find date format
    if not match:
        texts = re.sub(r'\d+', '', texts) # Remove numbers
        
    stop_words = set(stopwords.words('english'))
    filtered_text = ""
    words = texts.split()
    for word in words:
        if word.lower() not in stop_words:
            filtered_text += word + ' '   
    return filtered_text

Reading User stories from dataframe

In [86]:
sentences = df['user stories']

Cleaning the Data

In [98]:
SnowballStem = SnowballStemmer("english")
for i in range(0,len(df)):
    words = word_tokenize(clean_text(sentences[i]).strip())
    stemmed_words = [SnowballStem.stem(word) for word in words]
    sentences[i] = ' '.join(stemmed_words)

0                             want 12192017 delet process
1       want redesign resourc page match new broker de...
2       want report agenc user test awar contribut mak...
3       want move round 2 dab fab land page edit get a...
4       want move round 2 homepag edit get approv lead...
                              ...                        
1663                       want know zoonibot give explan
1664                        want know zoonibot say volunt
1665                           want group subject similar
1666    want recommend differ project volunt base prev...
1667                         want see summari articl reus
Name: user stories, Length: 1668, dtype: object